# SQL 的五十道練習

> 衍生計算欄位

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
import sqlite3
import unittest
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 從新冠肺炎每日報告資料表 `daily_report` 依據 `Confirmed`、`Recovered` 與 `Deaths` 衍生計算欄位 `Active`。

\begin{equation}
\text{Active} = \text{Confirmed} - \text{Recovered} - \text{Deaths}
\end{equation}

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3979, 4) 的查詢結果。

```
      Confirmed  Recovered  Deaths  Active
0         55023      47679    2400    4944
1         78127      47424    1380   29323
2        107339      73344    2891   31104
3          9937       9093     101     743
4         19796      18035     466    1295
...         ...        ...     ...     ...
3974       1817       1457      35     325
3975     158962     148993    1833    8136
3976       2121       1426     615      80
3977      54217      48000     763    5454
3978      33388      26044    1217    6127

[3979 rows x 4 columns]
```

In [2]:
calculate_active =\
"""
-- SQL 查詢語法起點
SELECT Confirmed,
       Recovered,
       Deaths,
       Confirmed - Recovered - Deaths AS Active
  FROM daily_report;
-- SQL 查詢語法終點
"""

## 從 NBA 球員資料表 `players` 依據身高 `heightMeters` 與 `weightKilograms` 衍生計算欄位 `bmi`。

\begin{equation}
BMI = \frac{weight_{kg}}{height_{m}^2}
\end{equation}

- 預期輸入：SQL 查詢語法。
- 預期輸出：(495, 3) 的查詢結果。

```
     heightMeters  weightKilograms        bmi
0            2.06            113.4  26.722594
1            2.01            108.0  26.732012
2            2.03            106.6  25.868136
3            2.08            120.2  27.782914
4            1.98             97.5  24.869911
..            ...              ...        ...
490          2.01            104.3  25.816193
491          2.08            106.1  24.523854
492          1.78             88.5  27.932079
493          1.98             90.7  23.135394
494          1.96             83.9  21.839858

[495 rows x 3 columns]
```

In [3]:
calculate_players_bmi =\
"""
-- SQL 查詢語法起點
SELECT heightMeters,
       weightKilograms,
       weightKilograms / (heightMeters*heightMeters) AS bmi
  FROM players;
-- SQL 查詢語法終點
"""

## 從 NBA 球隊資料表 `teams` 中連結 `divName` 與 `confName` 之後使用 `DISTINCT` 去除重複值。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(6, 1) 的查詢結果。

```
          conf_div
0  Southeast, East
1   Atlantic, East
2    Central, East
3  Southwest, West
4  Northwest, West
5    Pacific, West
```

In [4]:
concatenate_distinct_conference_division =\
"""
-- SQL 查詢語法起點
SELECT DISTINCT divName || ', ' || confName AS conf_div
  FROM teams;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All.

In [5]:
class TestCalculatedFields(unittest.TestCase):
    def test_calculate_active(self):
        active = pd.read_sql(calculate_active, conn)
        self.assertEqual(active.shape, (3979, 4))
        np.testing.assert_equal(active['Active'].values[:5],
                               np.array([4944, 29323, 31104, 743, 1295]))
        np.testing.assert_equal(active['Active'].values[-5:],
                               np.array([325, 8136, 80, 5454, 6127])) 
    def test_calculate_players_bmi(self):
        players_bmi = pd.read_sql(calculate_players_bmi, conn)
        self.assertEqual(players_bmi.shape, (495, 3))
        np.testing.assert_equal(players_bmi.columns.values,
                               np.array(['heightMeters', 'weightKilograms', 'bmi']))
    def test_concatenate_distinct_conference_division(self):
        distinct_conference_division = pd.read_sql(concatenate_distinct_conference_division, conn)
        self.assertEqual(distinct_conference_division.shape, (6, 1))
        np.testing.assert_equal(distinct_conference_division['conf_div'].values,
                               np.array(['Southeast, East', 'Atlantic, East', 'Central, East', 'Southwest, West', 'Northwest, West', 'Pacific, West']))

suite = unittest.TestLoader().loadTestsFromTestCase(TestCalculatedFields)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_calculate_active (__main__.TestCalculatedFields) ... ok
test_calculate_players_bmi (__main__.TestCalculatedFields) ... ok
test_concatenate_distinct_conference_division (__main__.TestCalculatedFields) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.194s

OK


In [6]:
print("您在 {} 道 SQL 練習中答對了 {} 題。".format(number_of_test_runs, number_of_successes))

您在 3 道 SQL 練習中答對了 3 題。
